In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler 
from collections import Counter

## Data Cleaning:
Data cleaning and preparation; admittedly, this is not the most efficient code -- but it will produce what we want.

In [2]:
#Read in dataframe
df = pd.read_csv("data_train.csv")

#Steps to clean data by dropping columns where the number of empty rows is >= 445,000
dfComplete = df.dropna(1, thresh= 445000)
# dfComplete.shape #Results in (494932,58)

#Steps to clean data by dropping row where the number of empty columns is > 0
dfCompleteAll = dfComplete.dropna(0, how="any")
# dfCompleteAll.shape #results in a dataframe of (476155,58)

# dfCompleteAll.isnull().sum() #no more nulls in the dataset

y = dfCompleteAll[["ASOURCE", "ATYPE", "RACE", "TOTCHG", "ZIPINC_QRTL"]]

In [3]:
"""The next step is to prepare variables for exploratory data analysis and feature
selection using a random forest. 

Random forest does not require standardization of continuous variables or normalization
of discrete variable. For categorical features, we will need to use pd.get_dummies or 
one hot encoding to create binary dummy variables. 
"""

#First create two dataframes of int and float values to make things easier to work with 
columnNames = dfCompleteAll.columns
dfFloat = pd.DataFrame()
dfInt = pd.DataFrame()
for name in columnNames:
    if dfCompleteAll[name].dtype == float:
        dfFloat = dfFloat.join(dfCompleteAll[name], how = "right")
    else:
        dfInt = dfInt.join(dfCompleteAll[name], how = "right")
        
#Convert all columns in DfFloat, except DISCWT, to integer values. Afterwards nominal features
#will be one-hot encoded to create dummy variables, again we will not normalize or stadardize
# numeric values. 

float_toInt = ['AGE', 'AMONTH', 'AWEEKEND', 'DIED', 'DISPUNIFORM', 'DXCCS1',
       'DXCCS2', 'FEMALE', 'LOS', 'PAY1', 'HOSP_BEDSIZE', 'HOSP_CONTROL',
       'HOSP_LOCTEACH']

for digit in float_toInt:
    dfFloat[digit] = dfFloat[digit].astype(int)
    
# In dfFloat we have the following columns and feature groupings. 
# For reference use feature_desc or call .unique() method on one of the columns

# CONTINUOUS:
dfFloatContinuous = dfFloat[["AGE", "DISCWT"]]


# NOMINAL:
dfFloatNominal = dfFloat[['AMONTH', 'DISPUNIFORM', 'DXCCS1',
       'DXCCS2', 'PAY1', 'HOSP_CONTROL','HOSP_LOCTEACH']]

# BINARY & ORDINAL
dfBinaryOrdinal = dfFloat[["DIED", "AWEEKEND", "FEMALE", "HOSP_BEDSIZE"]]

# DISCRETE
dfDiscrete = dfFloat[["LOS"]]

In [4]:
#Use pd.get_dummies to turn nominal variables into dummy variables by first setting all 
#values as string, a requirement of pd.get_dummies. 

dfFloatNominal = dfFloatNominal.loc[:].astype(str)
    
dfFloatNominal = pd.get_dummies(dfFloatNominal)
dfFloatNominal.shape 

(475155, 556)

In [5]:
# We can now recreate the original dfFloat dataframe as dfFloatPreprocessed which 
# will have variables ready for feature selection with RF. Next the same thing must be down
#with dfInt
list_of_dataframes = [dfFloatContinuous, dfBinaryOrdinal, dfDiscrete, dfFloatNominal]

dfFloatPreprocessed = pd.DataFrame()
for frame in list_of_dataframes:
    dfFloatPreprocessed = dfFloatPreprocessed.join(frame, how = "right")
dfFloatPreprocessed.shape

(475155, 563)

In [6]:
#Prepare dfInt for preprocessing, starting with dropping respone variables
dfInt = dfInt.drop(["ASOURCE", "ATYPE", "RACE", "TOTCHG", "ZIPINC_QRTL"], axis= 1)

#CMs are all binary, therefore, create a separate dataframe for these columns: 
dfCm = dfInt[['CM_AIDS', 'CM_ALCOHOL', 'CM_ANEMDEF',
       'CM_ARTH', 'CM_BLDLOSS', 'CM_CHF', 'CM_CHRNLUNG', 'CM_COAG',
       'CM_DEPRESS', 'CM_DM', 'CM_DMCX', 'CM_DRUG', 'CM_HTN_C', 'CM_HYPOTHY',
       'CM_LIVER', 'CM_LYMPH', 'CM_LYTES', 'CM_METS', 'CM_NEURO', 'CM_OBESE',
       'CM_PARA', 'CM_PERIVASC', 'CM_PSYCH', 'CM_PULMCIRC', 'CM_RENLFAIL',
       'CM_TUMOR', 'CM_ULCER', 'CM_VALVE', 'CM_WGHTLOSS']]

#Update the dfInt dataframe to a new dataframe:
dfIntShort = dfInt.drop(['CM_AIDS', 'CM_ALCOHOL', 'CM_ANEMDEF',
       'CM_ARTH', 'CM_BLDLOSS', 'CM_CHF', 'CM_CHRNLUNG', 'CM_COAG',
       'CM_DEPRESS', 'CM_DM', 'CM_DMCX', 'CM_DRUG', 'CM_HTN_C', 'CM_HYPOTHY',
       'CM_LIVER', 'CM_LYMPH', 'CM_LYTES', 'CM_METS', 'CM_NEURO', 'CM_OBESE',
       'CM_PARA', 'CM_PERIVASC', 'CM_PSYCH', 'CM_PULMCIRC', 'CM_RENLFAIL',
       'CM_TUMOR', 'CM_ULCER', 'CM_VALVE', 'CM_WGHTLOSS'],axis = 1)

#Update the dfInt dataframe to a new dataframe:
dfIntShort = dfInt.drop(['CM_AIDS', 'CM_ALCOHOL', 'CM_ANEMDEF',
       'CM_ARTH', 'CM_BLDLOSS', 'CM_CHF', 'CM_CHRNLUNG', 'CM_COAG',
       'CM_DEPRESS', 'CM_DM', 'CM_DMCX', 'CM_DRUG', 'CM_HTN_C', 'CM_HYPOTHY',
       'CM_LIVER', 'CM_LYMPH', 'CM_LYTES', 'CM_METS', 'CM_NEURO', 'CM_OBESE',
       'CM_PARA', 'CM_PERIVASC', 'CM_PSYCH', 'CM_PULMCIRC', 'CM_RENLFAIL',
       'CM_TUMOR', 'CM_ULCER', 'CM_VALVE', 'CM_WGHTLOSS'],axis = 1)

#Since NDX, NPR, ORPROC, TOTAL_DISC are all either Binary or Discerete variables, create a separate dataframe
dfIntBinaryDiscrete = dfIntShort[["NDX", "NPR", "ORPROC", "TOTAL_DISC"]]

# Since DQTR, HOSPID, MDC, NIS_STRATUM, HOSP_REGION are all nominal variables, create a separate dataframe to 
#turn these into dummy variables. ALSO Drop "KEY" as this is the record id field: 
dfIntToDummies = dfIntShort.drop(["KEY", "NDX", "NPR", "ORPROC", "TOTAL_DISC"], axis= 1)

#Turn values in DQTR, HOSPID, MDC, NIS_STRATUM, HOSP_REGION to string
dfIntToDummies = dfIntToDummies.loc[:].astype(str)

#Use pd.get_dummies to turn nominal string values to binary dummy variables
dfIntToDummies = pd.get_dummies(dfIntToDummies)
# dfIntToDummies.head()

intRecombine = [dfIntToDummies, dfCm, dfIntBinaryDiscrete]

dfIntPreprocessed = pd.DataFrame()
for df in intRecombine:
    dfIntPreprocessed = dfIntPreprocessed.join(df, how = "right")
    
dfPreprocessed = dfFloatPreprocessed.join(dfIntPreprocessed, how = "right")

In [7]:
#plug these dfs back together
dfFull = pd.concat([dfPreprocessed, y], axis=1)

## Check Outcome Variable Distribution

In [8]:
#first, separate our variables
X = dfFull.drop(['ATYPE'], axis=1)
y = dfFull.ATYPE

In [9]:
y.value_counts()

1    255366
3    106770
2     87084
4     24558
6       705
5       672
Name: ATYPE, dtype: int64

In [10]:
y.value_counts(normalize=True)

1    0.537437
3    0.224706
2    0.183275
4    0.051684
6    0.001484
5    0.001414
Name: ATYPE, dtype: float64

These class distributions are **highly imbalanced. We may want to consider oversampling our data with synthetic data creation packages.** But first, let's make a train/test split so regardless of what we do with oversampling, there's no information bleed. 

Then let's fit a "base" model random forest on the full data so we can compare subsequent models.

### Train/Test Split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 12)

## "Base" Model Random Forest
- In reality, this base model isn't at all a blind stab; we have methodically tuned Random Forests to predict other outcome variables in this dataset, and **we're starting with ideal parameters for models that imputed RACE, ASOURCE, etc.** (See the accompanying .ipynb "HCUP_Imputation_RF_Undersample_RACE_ZIPINCQRTL" for details).
- We want to see if we can beat this base model by either/both:
    - Oversampling small classes
    - Additional tuning to find new ideal parameters for this variable's distribution.

In [12]:
base_rfc = RandomForestClassifier(n_jobs=-1,max_features=100,n_estimators=300,max_depth=250,min_samples_split=2,
                             min_samples_leaf=1,oob_score = True,random_state=12)

base_rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=250, max_features=100, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=True, random_state=12, verbose=0, warm_start=False)

In [13]:
base_rfc_predictions = base_rfc.predict(X_test)
base_rfc_accuracy = accuracy_score(y_test, base_rfc_predictions)
base_rfc_accuracy

0.89542359861518872

In [14]:
base_rfc_report = classification_report(y_test, base_rfc_predictions)
print(base_rfc_report)

             precision    recall  f1-score   support

          1       0.91      0.95      0.93     51061
          2       0.84      0.76      0.80     17497
          3       0.89      0.85      0.87     21231
          4       0.99      1.00      1.00      4977
          5       0.84      0.47      0.60       132
          6       0.84      0.56      0.67       133

avg / total       0.89      0.90      0.89     95031



In [15]:
pd.crosstab(y_test, base_rfc_predictions, 
            rownames=['Actual ATYPE'], colnames=['Predicted ATYPE'])

Predicted ATYPE,1,2,3,4,5,6
Actual ATYPE,,,,,,
1,48688,1259,1098,2,11,3
2,3019,13256,1208,12,1,1
3,1865,1298,18047,11,0,10
4,3,7,1,4966,0,0
5,69,0,1,0,62,0
6,24,28,7,0,0,74


**This model does shockingly well.** So well that I had to double check that I didn't accidentally train it on test data.

Still, let's see if we can tune it at all.

### Randomly Sample a Subset of Our Data
- I have an early 2010s MacBook Air, so I can't run a grid search on 300k observation datasets.
- We'll do a grid search on this smaller subset of the data to find ideal params. Then we'll try them on our full data.
- The **RISK** is that in a smaller sample, there are so few class 4 and 5 observations that we're tuning a model based on what is effectively a different dataset. That's why we instantiated a base model (which really did quite well) to compare.

In [16]:
sample = dfFull.sample(n=30000)
X_sample = sample.drop(['ATYPE'], axis=1)
y_sample = sample.ATYPE

X_sample_train, X_sample_test, y_sample_train, y_sample_test = train_test_split(X_sample, y_sample, 
                                                                                test_size = 0.25, random_state=12)

In [17]:
#yeah, worried that 6 and 5 are so sparse that our tuning will be biased. We'll see!
y_sample_train.value_counts()

1    12061
3     4987
2     4154
4     1231
6       41
5       26
Name: ATYPE, dtype: int64

In [19]:
#pick our params; from (bad) experience, my machine can handle about 7 different params
param_grid = {
    'min_samples_split': [2, 5, 10, 25],
    'max_features': ['auto', 100, 300]
}

rfc_CV = GridSearchCV(estimator=base_rfc, param_grid=param_grid, scoring='f1_micro', 
                      n_jobs=-1, return_train_score=False)

rfc_CV.fit(X_sample_train, y_sample_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=250, max_features=100, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=True, random_state=12, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'min_samples_split': [2, 5, 10, 25], 'max_features': ['auto', 100, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='f1_micro', verbose=0)

In [20]:
rfc_CV.best_params_

{'max_features': 300, 'min_samples_split': 5}

In [25]:
rfc_CV_predictions = rfc_CV.predict(X_test)
rfc_CV_accuracy = accuracy_score(y_test, rfc_CV_predictions)
rfc_CV_accuracy

0.88228051898854054

Not surprisingly, this is worse.

In [26]:
rfc_CV_report = classification_report(y_test, rfc_CV_predictions)
print(rfc_CV_report)

             precision    recall  f1-score   support

          1       0.90      0.94      0.92     51061
          2       0.83      0.73      0.78     17497
          3       0.86      0.84      0.85     21231
          4       0.99      1.00      1.00      4977
          5       1.00      0.07      0.13       132
          6       0.81      0.56      0.66       133

avg / total       0.88      0.88      0.88     95031



No prediction improvement over the base model, because this model hasn't seen enough data.

## "Best Params" Random Forest
- Remember, it's entirely possible this will not improve results, as the CV was biased by computational necessity.
- Then again, it could have provided valuable insight. We'll see!

In [29]:
#unfortunately, this times out at these settings unless we reduce the training set; guess 300 features is too much
#this will still be generalizable - we are holding out a BIGGER test set after all
#we just hope our training data at 55% of the dataset is enough

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, y, test_size = 0.45, random_state = 12)

rfc_tuned = RandomForestClassifier(n_jobs=-1,max_features=300,n_estimators=300,max_depth=250,min_samples_split=5,
                             min_samples_leaf=1,oob_score = True,random_state=12)

rfc_tuned.fit(X_train_new, y_train_new)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=250, max_features=300, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=True, random_state=12, verbose=0, warm_start=False)

In [30]:
rfc_tuned_predictions = rfc_tuned.predict(X_test)
rfc_tuned_accuracy = accuracy_score(y_test, rfc_tuned_predictions)

array([3, 1, 3, ..., 3, 1, 1])

In [31]:
rfc_tuned_accuracy

0.89452915364460017

In [33]:
rfc_tuned_report = classification_report(y_test, rfc_tuned_predictions)
print(rfc_tuned_report)

             precision    recall  f1-score   support

          1       0.91      0.95      0.93     51061
          2       0.83      0.76      0.79     17497
          3       0.88      0.85      0.87     21231
          4       1.00      1.00      1.00      4977
          5       0.79      0.52      0.62       132
          6       0.84      0.55      0.66       133

avg / total       0.89      0.89      0.89     95031



## Model Comparison:

Strictly according to F1 scores, the tuned model has not improved upon our "base" model. We see some slightly lower precision/recall scores (.01) that are very likely just random chance. However, **it has distinguishing characteristics that could make it useful, depending on domain needs.**.
- The tuned model has *lower* precision but *higher* recall on class = 5. It is predicting 5 more often. It results in a lower f1 score, but depending on the nature of a problem, one sometimes wants to sacrifice precision for recall. We don't have the domain knowledge in health care research to know here, but it's good to have a choice in models.
- We also suspect that if, computationally, we were allow this tuned model to "see" more data that it would perform even better.

**"Tuned" Model:**

             precision    recall  f1-score   support

          1       0.91      0.95      0.93     51061
          2       0.83      0.76      0.79     17497
          3       0.88      0.85      0.87     21231
          4       1.00      1.00      1.00      4977
          5       0.79      0.52      0.62       132
          6       0.84      0.55      0.66       133

    avg / total       0.89      0.89      0.89     95031

**"Base" Model:**

                precision    recall  f1-score   support

          1       0.91      0.95      0.93     51061
          2       0.84      0.76      0.80     17497
          3       0.89      0.85      0.87     21231
          4       0.99      1.00      1.00      4977
          5       0.84      0.47      0.60       132
          6       0.84      0.56      0.67       133

    avg / total       0.89      0.90      0.89     95031